In [ ]:
from ccount import *
from os import environ

#todo: faster list comprehension of cropping
- 0627: reading new 2019 formatted czi images. very large, four scanned regions, zeros else-where. cut black regions, leave some margins for stitched images, put them together


```
from:
    0 
1       2
    3



to:
0 1
2 3
```

## Parameters

In [ ]:
# you can adjust these parameters to suit your data
# for run.sh
format="2019"
if environ.get('fname') is not None:
    fname = environ['fname']  # for runipy
else:
    # for notebook running
    fname = '../../ccount_data/E2F4_CFUe_14JUN19_stitching/C_CFUe_1-Stitching-10.czi'
    #fname = '../ccount_data/E2F4_CFUe_14JUN19/A_CFUe_1.czi'


    
    #fname = '../test_data/S0_0MMC_c.czi'



print('fname:', fname)

if format=='2019':
    block_height = 2000
    block_width = 2400 # pixcels, if 0, use whole image as block (still buggy whole image equalization)
    scaling_factor = 4 # 1: original dimension, 2: 1/2, 4: 1/4
    

In [ ]:
# please don't change unless necessary
# Jan 2020, jpg -> png; save images in workdir
# Jan 2020, scaling_factor=1, larger num_sigma 5->10, min_sigma 11->6
# Jan 2020, 
blob_extention_ratio = 1.4 # soft extend blob radius manually (1.4)
blob_extention_radius = 2 # pixcels to extend (2)
crop_width = 80  # padding width, which is cropped img width/2 (50)
overlap=.0

if scaling_factor == 1:
    max_sigma=50 
    min_sigma=4
    num_sigma=15
    threshold=0.1
elif scaling_factor == 2:
    max_sigma=40
    min_sigma=2
    num_sigma=10
    threshold=0.1
elif scaling_factor == 4:
    max_sigma=20
    min_sigma=8
    num_sigma=20  # 19Aug, 10 better sensitivity than 5, 1/2 speed
    threshold=0.02

    
outname = os.path.basename(fname)
detected_dbname = re.sub('.czi$', '.npy', outname)
labeled_dbname = re.sub('.czi$', '.labeled.npy', outname)
out_img_fname = re.sub('.czi$', '.png', outname)
equ_img_fname = re.sub('.czi$', '.equ.png', outname)

visualization = True
equalization = True

- min_sigmascalar or sequence of scalars, optional
the minimum standard deviation for Gaussian kernel. Keep this low to detect smaller blobs. The standard deviations of the Gaussian filter are given for each axis as a sequence, or as a single number, in which case it is equal for all axes.
- max_sigmascalar or sequence of scalars, optional
The maximum standard deviation for Gaussian kernel. Keep this high to detect larger blobs. The standard deviations of the Gaussian filter are given for each axis as a sequence, or as a single number, in which case it is equal for all axes.
- num_sigmaint, optional
The number of intermediate values of standard deviations to consider between min_sigma and max_sigma.
- thresholdfloat, optional.
The absolute lower bound for scale space maxima. Local maxima smaller than thresh are ignored. Reduce this to detect blobs with less intensities.

## Read whole image (czi)

In [ ]:
print(fname)
image = read_czi(fname, format="2019")
#image = down_scale(image, 4) # test
image = np.divide(image, np.max(image))  # from 0-255 or any range -> 0-1

# image = image[0:2048*3, 0:2048*3]

dims = np.divide(image.shape, 128) # out jpg size: 256 big, 512 mid, 1024 small
# finished 06/27

In [ ]:
# Test to speed up development
# image = down_scale(image, 4)
# image = image[0:image.shape[0]//8, 0:image.shape[1]//8]

In [ ]:
print(image.shape)

## Plot whole image (takes time)

In [ ]:
# ## Input Image
# if visualization:
#     fig, axes = plt.subplots(1,1, figsize=(20,20))
#     plt.imshow(image, 'gray')
#     plt.show()

## Find Blobs

In [ ]:
# detect with original image
if equalization:
    ## equalizagion
    print(">>> Equalizing image")
    if block_width <=0:
        print("Equalizing the whole image")
        image_equ = equalize(image)
    else:
        print("Equalizing image block by block")
        image_equ = block_equalize(image, block_height=block_height, block_width=block_width)
    
    ## visualization
#    if visualization:
#        print(">> Visualizing equalized image")
#        fig, axes = plt.subplots(1, 1, figsize=(20, 20), sharex=True, sharey=True)
#        ax[0].imshow(image_equ, 'gray')
#        plt.show()
#        #plt.savefig(equ_img_fname)
#        #print("equ_img saved as ", equ_img_fname)
    
    ## find blobs
    print(">>> Detecting blobs after equalization")
    crops = find_blobss_and_crop(
        image, image_equ,
        crop_width=crop_width, 
        # blob_detection parameters
        scaling_factor=scaling_factor,
        max_sigma=max_sigma, 
        min_sigma=min_sigma, 
        num_sigma=num_sigma, 
        threshold=threshold, 
        overlap=overlap,

        # blob yellow circle visualization parameters
        blob_extention_ratio=blob_extention_ratio,
        blob_extention_radius=blob_extention_radius
    )
else:
    ## find blobs on original photo
    print(">>> Detecting blobs without equalization")
    crops = find_blobss_and_crop(
        image, image,
        crop_width=crop_width, 
        # blob_detection parameters
        scaling_factor=scaling_factor,
        max_sigma=max_sigma, 
        min_sigma=min_sigma, 
        num_sigma=num_sigma, 
        threshold=threshold, 
        overlap=overlap,

        # blob yellow circle visualization parameters
        blob_extention_ratio=blob_extention_ratio,
        blob_extention_radius=blob_extention_radius
    )

In [ ]:
## Histogtam of radius
r_ = crops[:,2]
plt.hist(r_, 40)
plt.title("Histogram of blob size")
plt.show()

In [ ]:
## show randome blobs from all blobs
show_rand_crops(crops=crops, label_filter='na', num_shown=5, 
               blob_extention_ratio=blob_extention_ratio, blob_extention_radius=blob_extention_radius)

In [ ]:
## show blobs of specific sizes
filtered_blobs = filter_blobs(blobs=crops, r_min=80, r_max=80)
r_ = filtered_blobs[:,2]
plt.hist(r_, 40)
plt.title("Histogram of filtered blob size")
plt.show()
show_rand_crops(crops=filtered_blobs, label_filter='na', num_shown=5,
                blob_extention_ratio=blob_extention_ratio, blob_extention_radius=blob_extention_radius)

## Save all blobs detected (for labeling/training)

In [ ]:
print('there are {} blobs detected'.format(len(crops)))
print(crops.shape)
np.save(detected_dbname, crops)
print('saved into {}'.format(detected_dbname))

## Save html

In [ ]:
## save this file in html format
!jupyter nbconvert --to html blob_detection.ipynb


In [ ]:
%%bash
#mv blob_detection.html 1.blob_detection.large_block_equ.html